# Task 1: Pose Estimation

The function **estimate_pose** takes as input each instance of camera data and outputs the estimated position and euler angles of the quadrotor. The pose is obtained by solving the Perspective-n-Point (PnP) problem using the world and image points of detected corners of the april tags. If no tag ids are available in the camera data then empty arrays are returned by the function. A breif overview of my approach for camera pose estimation is given below:

**1. Loading and Extracting Necessary Data:**

Firstly, I load the data from a provided .mat file to extract camera sensor data - april tag ids, corner points in image frame and ground truth data - position, Euler angles, and timestamps.

**2. Image and World Point Calculation:**

Then, I calculate the world coordinates of the corner points of April tags based on their IDs using the **getWorldPoints** function. This involves using predefined map of tag IDs as shown below.

```python
# Map of april tag IDs 
tag_ids = np.array([[0, 12, 24, 36, 48, 60, 72, 84, 96],
                    [1, 13, 25, 37, 49, 61, 73, 85, 97],
                    [2, 14, 26, 38, 50, 62, 74, 86, 98],
                    [3, 15, 27, 39, 51, 63, 75, 87, 99],
                    [4, 16, 28, 40, 52, 64, 76, 88, 100],
                    [5, 17, 29, 41, 53, 65, 77, 89, 101],
                    [6, 18, 30, 42, 54, 66, 78, 90, 102],
                    [7, 19, 31, 43, 55, 67, 79, 91, 103],
                    [8, 20, 32, 44, 56, 68, 80, 92, 104],
                    [9, 21, 33, 45, 57, 69, 81, 93, 105],
                    [10, 22, 34, 46, 58, 70, 82, 94, 106],
                    [11, 23, 35, 47, 59, 71, 83, 95, 107]])
```

For a given tag id, I extract the row and column index of its position in the above map array. Based on the position in the array I calculate the world co-ordinates of the corner points of the tag.

It is given that - 'Each tag is a 0.152 m square with 0.152 m between tags, except for the space between columns 3 and 4, and 6 and 7, which is 0.178 m'. So the x and y co-cordinate for the corner p2 can be written as:

```python
p2x = 0.152*(2*row+1)
p2y = 0.152*(2*col+1) + (0.178 - 0.152)*(col//3)
```

The term (0.178 - 0152)*(col//3) takes care of the extra space between columns 3 and 4, and 6 and 7. After obtaining p2 corner co-ordinates, calculating the other corners is straightforward as shown below:

```python
p1x = p2x
p1y = p2y - 0.152

p3x = p1x - 0.152
p3y = p1y + 0.152

p4x = p1x - 0.152
p4y = p1y
```
The obtained corner points are packed into an array. The image co-ordinates for the corner points are obtained from the camera sensor data - p1,p2,p3,p4. I rearrange the corner points into a suitable format for OpenCV functions.

**3. Obtaining Camera Pose:**

I then obtain the camera pose by passing the image co-ordinates, world co-ordinates of corner points along with the given camera matrix and distortion parameters to the **cv2.solvePnP()** function.

$$ \text{camera\_matrix} = \begin{bmatrix}
314.1779 & 0 & 199.4848 \\
0 & 314.2218 & 113.7838 \\
0 & 0 & 1 \\
\end{bmatrix} $$

$$ \text{distortion\_params} = \begin{bmatrix}
-0.438607 & 0.248625 & 0.00072 & -0.000476 & -0.0911 \\
\end{bmatrix} $$

The solvePnP() function outputs a translation and rotation vector for the world frame w.r.t the camera frame.

**4. Transformation:**

After that, I transform the obtained camera pose into the robot/IMU frame pose w.r.t the world frame using the **cam2robotframe** function. The camera frame and robot/IMU frame are shown in the picture below. 

![Camera and robot frame configuration](/home/dheeraj/robot-navigation/NonlinearKalmanFilter/frames.png)

Based on the image above the transformation matrix of the robot/imu frame w.r.t the camera frame $T_{c}^{r}$ can be written as:

$$
T_{c}^{r} = \begin{pmatrix}
\cos\left(\frac{\pi}{4}\right) & -\cos\left(\frac{\pi}{4}\right) & 0 & -0.04 \\
-\sin\left(\frac{\pi}{4}\right) & -\sin\left(\frac{\pi}{4}\right) & 0 & 0.0 \\
0 & 0 & -1 & -0.03 \\
0 & 0 & 0 & 1 \\
\end{pmatrix}
$$

Next I calculate the transformation matrix of the world frame w.r.t camera frame $T_{c}^{w}$ from the translation and rotation vectors obtianed by the solvePnP() function. The rotation matrix is obtained by passing the rotation vector to **cv2.rodriguez()** function.

The final transformation of the robot/imu frame w.r.t world frame is obtained by $$ T_{w}^{r} = T_{c}^{w^{-1}} \cdot T_{c}^{r} $$


I repeat this process for all camera data instances, generating position and orientation estimates along with corresponding timestamps.

**Challenges:** 

1. Empty April tag IDs: In some instances there are no april tags detected in the image and the tag id array is empty. Hence the camera pose cannot be estimated in such cases. So I set the position and euler angles as NaN whenever the pose cannot be estimated. 

# Task 2: Trajectory Visualization

Pose estimation is carried out for each given dataset and the results are plotted against the ground truth Vicon data. The **visualize_trajectory** function takes as input the dataset filename and plots the estimated poses against the ground truths. The function plots three figures, first is the 3D plot of the estimated position vs ground truth position, second figure consists of individual subplots of position co-ordinates (this was not required but it helps visualize how close the position estimates are to the ground truths) and the third figure is the subplots of the euler-angles.

Below are the trajectory visualization results for each dataset:

## Dataset 0

In [ ]:
from pose_estimation import *

file_name = 'studentdata0.mat'
visualize_trajectory(file_name)

**Discussion:**

1. 3D Position: The estimated trajectory mirrors the Vicon reference closely, which demonstrates good estimation accuracy. There are some divergences along the path, but the overall shapes are similar.

2. Position: The estimated positions closely follow the Vicon data with high correlation, suggesting accurate estimation.

3. Euler Angles: The estimated measurements have slight deviations, indicating nosiy estimations, especially in roll and pitch. Also there is an noticeable offset in the yaw angle.

## Dataset 1

In [ ]:
file_name = 'studentdata1.mat'
visualize_trajectory(file_name)


**Discussion:**
1. The 3D position plot reveals significant divergence between the estimated and reference trajectories, with the estimated path showing more variance and less smoothness.

2. In the position over time plot, the X and Y estimations have occasional deviations, while the Z estimation diverges more noticeably after around 12.5 units of time, indicating less reliable estimation in this axis.

3. The Euler Angles plot indicates a good estimation of yaw over time with minor divergence, while roll and pitch estimations show increased noise and deviation, suggesting less stability in these estimations.

## Dataset 2

In [ ]:
file_name = 'studentdata2.mat'
visualize_trajectory(file_name)

**Discussion:**

1. The estimated 3D position has large variances and often diverges significantly from the reference, suggesting a less accurate estimation process or errors in the estimation algorithm.

2. The X and Y position estimations show high-frequency fluctuations and short-term divergences from the reference, with X estimation showing greater deviation. The Z estimation, however, matches the reference with higher accuracy except for some peaks, demonstrating some consistency.

3. For the Euler angles, the roll estimation is particularly noisy, with pitch estimation showing some high peaks of deviation. The yaw estimation is the most accurate, with both estimations showing similar trends but with slight offsets and noise throughout.

## Dataset 3

In [ ]:
file_name = 'studentdata3.mat'
visualize_trajectory(file_name)

**Discussion:**
1. The 3D position comparison shows considerable discrepancies between the estimated and reference paths, with the estimated path being less smooth and more erratic.

2. For the position over time, the X estimation is quite erratic and shows many peaks diverging from the reference. The Y estimation, while following the same general pattern as the reference, exhibits several peaks and troughs. The Z estimation follows the reference but with a significant offset.

3. Regarding Euler angles, the pitch estimation is highly erratic with frequent deviations. The roll estimation, despite following the reference curve, has multiple instances of high deviation. The yaw estimation has a good correlation with the reference.

## Dataset 4

In [ ]:
file_name = 'studentdata4.mat'
visualize_trajectory(file_name)

**Discussion:**

1. The estimated 3D position shows large discrepancies from the reference, with much greater variance and multiple loops diverging from the intended path.However the individual subplots show that the position do slightly follow the vicon groundtruths.

2. The X,Y and Z position estimates show some sporadic peaks that do not align with the reference around time=10s. The Z position estimation follows the reference but with a notable offset.

3. The roll and pitch estimations exhibit a high degree of noise with some sporadic peaks that do not align with the reference around time=10s similar tot he position subplots. The yaw estimation is closely aligned with the gorundtruths.

## Dataset 5

In [ ]:
file_name = 'studentdata5.mat'
visualize_trajectory(file_name)

**Discussion:**

1. The 3D position comparison reveals a clear trend discrepancy with the estimated path significantly deviating from the reference, particularly in the Z dimension.

2. In the position plots, the X and Y estimation is mostly in agreement with the reference, whereas the Z estimations have a significant offset

3. For the Euler angles, roll and pitch estimations are quite noisy and show multiple spikes, while the yaw estimation shows a closer match to the reference gorundtruths but they still have deviations.

## Dataset 6

In [ ]:
file_name = 'studentdata6.mat'
visualize_trajectory(file_name)

**Discussion:**

The results are quite similar to the previous dataset.

1. The 3D plot shows that there is a significant offset in the estimated trajectory in the Z dimension.

2. X and Y position estimates closely align with the reference. However there is an offset seen in Z postion estimates.

3. For Euler angles, roll and pitch esimtates show notable spikes with yaw estimation being better but still noisy.

## Dataset 7

In [ ]:
file_name = 'studentdata7.mat'
visualize_trajectory(file_name)

**Discussion:**

Results very similar to the two previous datasets. A significant offset is present in the Z position estimates. Roll and Pitch angle estimates have high frequency noise with yaw angle having less fluctuations but still off from the reference gorundtruths.

# Task 3: Covariance Estimation

The timestamps from the camera data are not synchronized with those from the ground truth. To compute the covariance of the pose estimates, I apply linear interpolation to the estimated pose data, aligning it with the ground truth timestamps, under the assumption that the timestamps are sufficiently proximate for this method to be valid. 

However, there are instances where the pose estimate returns as NaN due to the camera's failure to detect April tags. To address this, I filter out the NaN values from the pose estimation data by excluding the corresponding timestamp before proceeding with the interpolation. The interpolation is performed by the **scipy.interpolate.interp1d()** function. $v_t$ is obtained by subtracting the interpolated estimated pose data with the ground truth vicon data at each groundtruth timestamp. The covarinace matrix is then calculated using the below formula: $$ \mathbf{R} = \frac{1}{n-1} \sum_{t=1}^{n} \mathbf{v}_t \mathbf{v}_t^T $$

The function **estimate_covariance** implements the above approach and the covariance matrices calculated for each data set are shown below. The covariance matrix values are rounded up to 5 decimal places which should give enough precision as the position can be accurate to 1 mm which is 1e-3m and the angle can be accurate to 1 deg which is 0.0175 rad.

## Dataset 0

In [ ]:
file_name = 'studentdata0.mat'
R = estimate_covariances(file_name)
print(R)

## Dataset 1

In [ ]:
file_name = 'studentdata1.mat'
R = estimate_covariances(file_name)
print(R)

## Dataset 2

In [ ]:
file_name = 'studentdata2.mat'
R = estimate_covariances(file_name)
print(R)

## Dataset 3

In [ ]:
file_name = 'studentdata3.mat'
R = estimate_covariances(file_name)
print(R)

## Dataset 4

In [ ]:
file_name = 'studentdata4.mat'
R = estimate_covariances(file_name)
print(R)

## Dataset 5

In [ ]:
file_name = 'studentdata5.mat'
R = estimate_covariances(file_name)
print(R)

## Dataset 6

In [ ]:
file_name = 'studentdata6.mat'
R = estimate_covariances(file_name)
print(R)

## Dataset 7

In [ ]:
file_name = 'studentdata7.mat'
R = estimate_covariances(file_name)
print(R)

# Task 4: Extended Kalman Filter Implementation

The class **EKF** implements the extended kalman filter algorithm. The EKF object is initialized with the length of the state and observation vector along with the symbolic process and observation models (functions are defined using sympy). During the initialization procedure the process model is differentiated with the symbolic state vector to obtain the state jacobian matrix A and similarly the observation model is differentiated symbolically to obtain the observation matrix C.

Next the process and measurement noise covariances, Q and R were set. The following values were used for the filter:

```python
# process noise
Q = 1e-4*np.eye(N_STATE)
# measurement noise
R = 1e-3*np.eye(N_OBS)
```

The intial state was set as the gorund truth vicon state at the 0th index with bais set as zero for both the gyroscope and accelerometer.

For a given dataset, the function **EtendedKalmanFilter()** iterates over the entire dataset estimating the states and plots the resulting pose against the ground truth vicon pose. For each instance in the dataset, given the timespan dt and the control inputs u, the methods predict() and update() of the EKF class are called to predict and update the states, respectively.  

The subsequent sections present the results of the EKF pose estimates for all provided datasets.

**Discussion**
1. The predict() method's evaluation of the symbolic process model and the Jacobian state matrix A for each iteration significantly slows down the Extended Kalman Filter (EKF). On average, the EKF takes 2 minutes and 30 seconds to process a single dataset. This underscores the computational expense of the EKF implementation. Nevertheless, it remains simpler and more straightforward to implement compared to the Unscented Kalman Filter (UKF).
2. Adjusting the measurement covariance presents a challenge, as the pose estimates vary significantly across different datasets. For instance, in the presented results, the EKF performs well in tracking the X coordinate in dataset0 but poorly in dataset1. This discrepancy indicates that the covariance for dataset1's X coordinate needs to be higher. Similar variations are observed across different datasets, where the EKF may excel in tracking one state variable in one dataset but exhibit inferior performance in others.

## Dataset0

In [ ]:
from ekf import ExtendedKalmanFilter

file_name = 'studentdata0.mat'
ExtendedKalmanFilter(file_name)

## Dataset1

In [ ]:
file_name = 'studentdata1.mat'
ExtendedKalmanFilter(file_name)

## Dataset2

In [ ]:
file_name = 'studentdata2.mat'
ExtendedKalmanFilter(file_name)

## Dataset3

In [ ]:
file_name = 'studentdata3.mat'
ExtendedKalmanFilter(file_name)

## Dataset4

In [ ]:
file_name = 'studentdata4.mat'
ExtendedKalmanFilter(file_name)

## Dataset5

In [ ]:
file_name = 'studentdata5.mat'
ExtendedKalmanFilter(file_name)

In [ ]:
file_name = 'studentdata6.mat'
ExtendedKalmanFilter(file_name)

In [ ]:
file_name = 'studentdata7.mat'
ExtendedKalmanFilter(file_name)